# For all runs

In [ ]:
import pandas as pd
import wandb
import numpy as np

api = wandb.Api()

runs = api.runs(
    path="explainableml/ICL_torch",
    filters={
        "config.progress_measure": True,
        "config.rope": True,
        "state": "finished",
        "Tags": "normalize_progress_measure"
    }
)
# Extract data and filter based on IC_Accuracy
# runs = [
#     run for run in runs 
#     if run.config.get("K", 0) in [2048,4096,8192,16384]]
# ---- 1. Gather every time-step from every run ---------------------------------
#    For each metric build ONE long Series by concatenating the histories
bags = {
    "IC_Accuracy":        [],
    "IC2_Accuracy":       [],
    "IH_strength_layer1": [],
    "IH_strength_layer2": [],
    "TILA_layer1":        [],
    "TILA_layer2":        [],
    "TIIA_layer1":        [],
    "TIIA_layer2":        [],
    "TLA_layer1":         [],
    "TLA_layer2":         [],
    "prev_1_attn_layer1": [],
    "prev_1_attn_layer2": [],
    "prob_icl_labels":    [],
}

for run in runs:
    h = run.history()                     # one DataFrame per run
    for k in bags:
        # Normalize the vector
        v = (h[k] - np.mean(h[k]).item()) / np.std(h[k]).item()
        bags[k].append(v)              # keep the whole Series (all steps)

# ---- 2. Concatenate into vectors ---------------------------------------------
vectors = {k: pd.concat(v, ignore_index=True) for k, v in bags.items()}

# ---- 3. Correlation matrix (Pearson, by default) -----------------------------
df = pd.DataFrame(vectors) 
corr = df.corr(method="pearson")

# ---- 4. Correlation of every metric with IC_Accuracy -------------------------
corr_with_ic = corr["IC_Accuracy"].sort_values(ascending=False)
corr_with_ic2 = corr["IC2_Accuracy"].sort_values(ascending=False)

print("\nPearson r between each metric vector and IC_Accuracy:")
print(corr_with_ic)
# print("\nPearson r between each metric vector and IC2_Accuracy:")
# print(corr_with_ic2)
